# LSTM Experiments


In [1]:
import pandas
import numpy

C:\Users\joehi\AppData\Local\Temp\ipykernel_9464\2583639460.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [4]:
BASE_PATH = r"C:\Users\joehi\Documents\Work\Uni - Year 4\COMP61332\Coursework\dataset\data\json"
# FILE = r"\train.json"
FILE = r"\dev.json"

df = pandas.read_json(BASE_PATH + FILE)
df.head(10)

,id,docid,relation,token,subj_start,subj_end,obj_start,obj_end,subj_type,obj_type,stanford_pos,stanford_ner,stanford_head,stanford_deprel
0,e7798fb926b9403cfcd2,APW_ENG_20101103.0539,per:title,"[At, the, same, time, ,, Chief, Financial, Off...",8,9,12,12,PERSON,TITLE,"[IN, DT, JJ, NN, ,, NNP, NNP, NNP, NNP, NNP, M...","[O, O, O, O, O, O, O, O, PERSON, PERSON, O, O,...","[4, 4, 4, 12, 12, 10, 10, 10, 10, 12, 12, 0, 1...","[case, det, amod, nmod, punct, compound, compo..."
1,e779865fb96bbbcc4ca4,APW_ENG_20080229.1401.LDC2009T13,no_relation,"[U.S., District, Court, Judge, Jeffrey, White,...",17,18,4,5,PERSON,PERSON,"[NNP, NNP, NNP, NNP, NNP, NNP, IN, NNP, VBD, D...","[LOCATION, O, O, O, PERSON, PERSON, O, O, O, O...","[6, 6, 6, 6, 6, 9, 8, 6, 0, 11, 9, 13, 11, 20,...","[compound, compound, compound, compound, compo..."
2,e7798ae9c0adbcdc81e7,APW_ENG_20090707.0488,per:city_of_death,"[PARIS, 2009-07-07, 11:07:32, UTC, French, med...",9,9,0,0,PERSON,CITY,"[NNP, CD, CD, NNP, NNP, NNS, RBR, VBD, IN, NNP...","[LOCATION, TIME, TIME, TIME, MISC, O, O, O, O,...","[6, 6, 6, 6, 6, 8, 8, 0, 16, 16, 10, 10, 12, 1...","[compound, nummod, nummod, compound, compound,..."
3,e779865fb9bddb4eccbc,LTW_ENG_20070522.0147.LDC2009T13,no_relation,"[The, current, holdings, of, Blackstone-operat...",13,14,1,1,ORGANIZATION,DATE,"[DT, JJ, NNS, IN, JJ, NNS, VBP, NNP, NNP, ,, N...","[O, DATE, O, O, MISC, O, O, ORGANIZATION, ORGA...","[3, 3, 7, 6, 6, 3, 0, 9, 7, 9, 12, 9, 9, 15, 9...","[det, amod, nsubj, case, amod, nmod, ROOT, com..."
4,e779865fb9d2787a87ff,XIN_ENG_20060809.0432.LDC2007T07,no_relation,"[The, ICDC, was, formed, after, the, Nepali, g...",25,27,6,6,PERSON,MISC,"[DT, NN, VBD, VBN, IN, DT, JJ, NN, CC, DT, NNS...","[O, ORGANIZATION, O, O, O, O, MISC, O, O, O, O...","[2, 4, 4, 0, 8, 8, 8, 4, 8, 11, 8, 11, 15, 15,...","[det, nsubjpass, auxpass, ROOT, case, det, amo..."
5,e779865fb97262ce4da3,eng-NG-31-108589-8120474,no_relation,[http://groups.yahoo.com/group/aspartameNM/mes...,27,27,53,53,ORGANIZATION,PERSON,"[NN, NN, IN, DT, JJ, JJ, NN, IN, NN, NN, :, NN...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 0, 7, 7, 7, 7, 2, 10, 10, 7, 2, 13, 2, 13,...","[compound, ROOT, case, det, amod, amod, nmod, ..."
6,e7798cd9013281a16790,WPB_ENG_20100119.0012,org:shareholders,"[Warren, Buffett, ,, who, committed, most, of,...",34,38,23,23,ORGANIZATION,PERSON,"[NNP, NNP, ,, WP, VBD, JJS, IN, PRP$, NNP, NNP...","[PERSON, PERSON, O, O, O, O, O, O, ORGANIZATIO...","[2, 17, 2, 5, 2, 5, 12, 12, 12, 12, 12, 6, 14,...","[compound, nsubj, punct, nsubj, acl:relcl, dob..."
7,e779865fb9abef08f14e,APW_ENG_20081024.1282.LDC2009T13,no_relation,"[``, She, made, a, devastating, run, around, t...",16,17,11,14,PERSON,ORGANIZATION,"[``, PRP, VBD, DT, JJ, NN, IN, DT, NN, ,, '', ...","[O, O, O, O, O, O, O, O, O, O, O, ORGANIZATION...","[19, 3, 19, 6, 6, 3, 9, 9, 3, 19, 19, 13, 19, ...","[punct, nsubj, ccomp, det, amod, dobj, case, d..."
8,e779837935f731fc9f0f,AFP_ENG_20100912.0191,per:origin,"[Prolific, French, film, maker, Claude, Chabro...",4,5,1,1,PERSON,NATIONALITY,"[JJ, JJ, NN, NN, NNP, NNP, ,, WP, VBD, VB, DT,...","[O, MISC, O, O, PERSON, PERSON, O, O, O, O, O,...","[4, 4, 4, 0, 6, 19, 6, 9, 6, 9, 14, 14, 14, 10...","[amod, amod, compound, ROOT, compound, nsubj, ..."
9,e7798ae9c0f371b45c48,NYT_ENG_20100818.0143,per:city_of_death,"[The, self-declared, mastermind, of, the, atta...",7,8,24,24,PERSON,CITY,"[DT, JJ, NN, IN, DT, NN, ,, NNP, NNP, ,, RB, V...","[O, O, O, O, O, O, O, PERSON, PERSON, O, O, O,...","[3, 3, 21, 6, 6, 3, 3, 9, 3, 3, 12, 3, 16, 16,...","[det, amod, nsubj, case, det, nmod, punct, com..."
